In [3]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

layout1 = pd.read_csv('F:/Entity_Matching/pythonProject/data/ABC_layout_1.csv')
layout2 = pd.read_csv('F:/Entity_Matching/pythonProject/data/PQR_layout_2.csv')
layout3 = pd.read_csv('F:/Entity_Matching/pythonProject/data/layout_3_voters.csv')
layout4 = pd.read_csv('F:/Entity_Matching/pythonProject/data/KLM_layout_4.csv')
layout5 = pd.read_csv('F:/Entity_Matching/pythonProject/data/PQR_layout_2.csv')

del layout1["Last Name"]
del layout2["Unnamed: 0"]
del layout4["Unnamed: 0"]

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

layout1_ = layout1.copy()
layout2_ = layout2.copy()
layout3_ = layout3.copy()
layout4_ = layout4.copy()
layout5_ = layout5.copy()


def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else x)

layout1_ = sanitize(layout1_)
layout2_ = sanitize(layout2_)
layout3_ = sanitize(layout3_)
layout4_ = sanitize(layout4_)
layout5_ = sanitize(layout5_)

soup1 = ['Name', 'Date of Birth', 'Father_Name']
soup2 = ['Name', 'Date of Birth', 'Father_Name']
soup3 = ['Name', 'Date of Birth', 'Father_Name']
soup4 = ['Name', 'Date of Birth', 'Father_Name']
soup5 = ['Name', 'Date of Birth', 'Father_Name']

def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

create_soup(layout1, layout1_, soup1, 'soup1')
create_soup(layout2, layout2_, soup2, 'soup2')
create_soup(layout3, layout3_, soup3, 'soup3')
create_soup(layout4, layout4_, soup4, 'soup4')
create_soup(layout5, layout5_, soup5, 'soup5')


def column_remover(df):

    columns = ["Name", "Date of Birth", "Father_Name", "Temporary_Address", "Mobile Number", "Permanent_Address",  "Mother Name"]
    column_pairs = [(col, f"{col}_x", f"{col}_y") for col in columns]

    for new_col, col_x, col_y in column_pairs:
        if col_x in df.columns and col_y in df.columns:
            df[new_col] = df[col_x].combine_first(df[col_y])
            df.drop([col_x, col_y], axis=1, inplace=True)

    return df

def combine_layouts(A, B, soup_A, soup_B, threshold=0):
    tfidf = TfidfVectorizer(stop_words='english')
    
    combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
    tfidf.fit(combined_soup)
    
    tfidf_matrix_A = tfidf.transform(A[soup_A])
    tfidf_matrix_B = tfidf.transform(B[soup_B])
    
    similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
    similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)

    max_idx_row = similarity_df.idxmax(axis=1)
    similarity_mask = similarity_df.max(axis=1) > threshold
    
    combined_df = pd.DataFrame({
        soup_A: A[soup_A].values,
        soup_B: [B.loc[idx, soup_B] if mask else None for idx, mask in zip(max_idx_row.values, similarity_mask)]
    })
    
    result = pd.merge(pd.merge(A, combined_df, on=soup_A, how='left'), B, on=soup_B,  how='inner')
    result.drop(columns = soup_B, inplace = True)
    column_remover(result)
    return result

result_12 = combine_layouts(layout1, layout2, 'soup1', 'soup2')
result_123 = combine_layouts(result_12, layout3, 'soup1', 'soup3')
result_1234 = combine_layouts(result_123, layout4, 'soup1', 'soup4')
final_result = combine_layouts(result_1234, layout5, 'soup1', 'soup5')

In [4]:
final_result